# Klasy

# Metody __new__ & __init__

In [55]:
class Dummy:
    def __new__(cls, *args):
        print(f"Dummy.__new__({cls}, {args}) has been called...")
        obj = super().__new__(cls) # calling __new__ from object
        obj.extra_attribute = "Extra Attribute"
        print(f"Object {obj} has been created...")
        return obj
    
    def __init__(self, *args):
        print(f"Object's __dict__: {self.__dict__}")
        print(f"Dummy.__init__({self}, {args})...")
        self.args = args
        print(f"Object's __dict__: {self.__dict__}")

In [56]:
d = Dummy(1, "One")

Dummy.__new__(<class '__main__.Dummy'>, (1, 'One')) has been called...
Object <__main__.Dummy object at 0x0000024D3872B8D0> has been created...
Object's __dict__: {'extra_attribute': 'Extra Attribute'}
Dummy.__init__(<__main__.Dummy object at 0x0000024D3872B8D0>, (1, 'One'))...
Object's __dict__: {'extra_attribute': 'Extra Attribute', 'args': (1, 'One')}


## Kiedy używać `__new__`?

In [57]:
class UppercaseTuple(tuple):
    def __init__(self, list) -> None:
        print(f"Start changes for {list}")

        for i, item in enumerate(list):
            self[i] = item.upper()

In [58]:
UppercaseTuple(["hello", "world", "!"])

Start changes for ['hello', 'world', '!']


TypeError: 'UppercaseTuple' object does not support item assignment

In [59]:
class UppercaseTuple(tuple):
    def __new__(cls, list):
        print(f"Start changes for {list}")
        new_content = [item.upper() for item in list]       
        return super().__new__(cls, new_content)

In [60]:
UppercaseTuple(["hello", "world", "!"])

Start changes for ['hello', 'world', '!']


('HELLO', 'WORLD', '!')